# This is how we extracted the first few hundred vids for dynamo ingestion


In [2]:
import numpy as np
import pandas as pd
import pickle

In [3]:
!ls ../../models
df = pd.read_pickle('/root/models/predictions.pickle')

predictions.pickle
simpleCNN_1000epochs_20191110_150328.h5
simpleCNN_1000epochs_20191110_153647.h5
simpleCNN_1000epochs_20191110_171702.h5
simpleCNN_1000epochs_20191112_024957.h5
simpleCNN_100epochs_20191110_033120.h5
simpleCNN_10epochs_20191109_192747.h5
simpleCNN_history_1000epochs_20191110_150328.pickle
simpleCNN_history_1000epochs_20191110_153647.pickle
simpleCNN_history_1000epochs_20191110_171702.pickle
simpleCNN_history_1000epochs_20191112_024957.pickle
simpleCNN_history_100epochs_20191110_033120.pickle
simpleCNN_history_10epochs_20191109_192747.pickle
test.pickle
train.pickle


In [75]:
df.set_index('video-UID',inplace=True)

In [76]:
df.drop('ohe_labels',inplace=True,axis=1)

In [77]:
df.columns

Index(['clip_title', 'labeler', 'label', 'y_prob', 'y_pred', 'predix'], dtype='object')

In [78]:
df['pred_class_proba']=df.apply(lambda x: x['y_prob'][x['predix']],axis=1)

In [79]:
df.head()

,clip_title,labeler,label,y_prob,y_pred,predix,pred_class_proba
video-UID,,,,,,,
2857,downward_strike_punches_slice655.mp4,Alex,0,"[0.91471887, 0.04957079, 3.3734228e-05, 0.0336...","[1, 0, 0, 0, 0]",0,0.914719
12907,downward_strike_punches_slice275.mp4,Jen,0,"[0.7162727, 0.13020208, 0.0011639277, 0.133737...","[1, 0, 0, 0, 0]",0,0.716273
5205,traditional_dance_slice415.mp4,Jen,0,"[0.76954263, 0.19615111, 0.008012304, 0.012842...","[1, 0, 0, 0, 0]",0,0.769543
12430,V_789_slice325.mp4,Ahsen,1,"[0.35030097, 0.64757156, 0.0005246858, 0.00062...","[0, 1, 0, 0, 0]",1,0.647572
13035,downward_strike_punches_slice850.mp4,Lance,0,"[0.73543257, 0.07232628, 1.3397551e-05, 0.1898...","[1, 0, 0, 0, 0]",0,0.735433


In [80]:
df.drop('y_prob',inplace=True,axis=1)

In [81]:
df.drop('y_pred',inplace=True,axis=1)

In [83]:
df['recent_play']=None

In [86]:
df['num_humans_saw_punch']=df['label'].apply(lambda x: 1 if x > 0 else 0)

In [87]:
df['num_humans_saw_none']= 1 - df['num_humans_saw_punch']

In [92]:
df['portion_humans_saw_punch']=df.apply(lambda x: np.divide(x['num_humans_saw_punch'],\
                            x['num_humans_saw_punch']+x['num_humans_saw_none']),axis=1)

In [95]:
df['portion_humans_saw_none']= 1 - df['portion_humans_saw_punch']

In [101]:
df['human_consensus']=df.apply(lambda x: 1 if x['num_humans_saw_punch']>x['num_humans_saw_none'] else 0,axis=1)

In [105]:
df.rename(columns={'label':'gt_class','predix':'pred_class'},inplace=True)

In [109]:
df.sample(5)

,clip_title,labeler,gt_class,pred_class,pred_class_proba,recent_play,num_humans_saw_punch,num_humans_saw_none,portion_humans_saw_punch,portion_humans_saw_none,human_consensus
video-UID,,,,,,,,,,,
13978,marine_punches_navy_corpsman_slice50.mp4,Ahsen,0,0,0.722466,None,0,1,0.0,1.0,0
4591,streetfight2_slice395.mp4,Alex,1,0,0.783940,None,1,0,1.0,0.0,1
14251,short_strikes_slice145.mp4,Alex,0,0,0.876232,None,0,1,0.0,1.0,0
14756,streetfight2_slice420.mp4,Ahsen,0,0,0.725681,None,0,1,0.0,1.0,0
11896,V_352.mp4,Alex,1,0,0.776145,None,1,0,1.0,0.0,1


In [110]:
df.to_csv("first_dynamo_load.csv")